In [7]:
import pandas as pd
import json
import os
import numpy as np

In [8]:
# first load prompt results
# {"idx": 3, "result": "[calculate_displacement(initial_velocity=15, acceleration=9.8, time=10)]", "input_token_count": 473, "output_token_count": 19, "latency": 0.47064781188964844}

error_results_df_list = []
full_results_df_list = []
acc_dict = {'model': [], 'filename': [], 'accuracy': [], 'correct_count': [], 'total_count': []}
for model in ['gpt-3.5-turbo-0125', 'gpt-3.5-turbo-0125-FC']:
    # model = 'gpt-3.5-turbo-0125'
    results_dir = f'score/{model}'
    json_files = [f'{results_dir}/{f}' for f in os.listdir(results_dir) if f.endswith('.json')]
    for filename in json_files:
        with open(filename, 'r') as f:
            try:
                data = [json.loads(line) for line in f.readlines()]
                # skip the accuracy line
                df = pd.DataFrame(data[1:])
                df['filename'] = filename.split('/')[-1]
                error_results_df_list.append(df)
                # parse out accuracy_info
                acc_info = data[0]
                acc_dict['filename'].append(filename.split('/')[-1])
                acc_info['model'] = model
                for key in acc_info.keys():
                    acc_dict[key].append(acc_info[key])
            except Exception as e:
                print(f'Error reading {filename}: {e}')

# now read full results
for model in ['gpt-3.5-turbo-0125', 'gpt-3.5-turbo-0125-FC']:
    results_dir = f'result/{model}'
    json_files = [f'{results_dir}/{f}' for f in os.listdir(results_dir) if f.endswith('.json')]
    for filename in json_files:
        with open(filename, 'r') as f:
            try:
                data = [json.loads(line) for line in f.readlines()]
                df = pd.DataFrame(data)
                df['filename'] = filename.split('/')[-1]
                df['model_name'] = model
                full_results_df_list.append(df)
            except Exception as e:
                print(f'Error reading {filename}: {e}')

acc_df = pd.DataFrame(acc_dict)
acc_df['metric'] = acc_df['filename'].apply(lambda x: x.split('/')[-1].split('.')[0])
error_result_df = pd.concat(error_results_df_list)
full_result_df = pd.concat(full_results_df_list)

Error reading score/gpt-3.5-turbo-0125-FC/executable_multiple_function_score_pallavi_annotated.json: Expecting value: line 1 column 1 (char 0)


In [9]:
acc_df[['model', 'filename', 'accuracy']].sort_values(by=['filename', 'model'])

,model,filename,accuracy
4,gpt-3.5-turbo-0125,executable_multiple_function_score.json,0.720000
16,gpt-3.5-turbo-0125-FC,executable_multiple_function_score.json,0.680000
10,gpt-3.5-turbo-0125,executable_parallel_function_score.json,0.620000
23,gpt-3.5-turbo-0125-FC,executable_parallel_function_score.json,0.720000
9,gpt-3.5-turbo-0125,executable_parallel_multiple_function_score.json,0.450000
12,gpt-3.5-turbo-0125-FC,executable_parallel_multiple_function_score.json,0.400000
2,gpt-3.5-turbo-0125,executable_simple_score.json,0.780000
22,gpt-3.5-turbo-0125-FC,executable_simple_score.json,0.750000
6,gpt-3.5-turbo-0125,java_score.json,0.510000
13,gpt-3.5-turbo-0125-FC,java_score.json,0.530000


In [53]:
for model in acc_df['model'].unique():
    acc = acc_df[acc_df['model'] == model].correct_count.sum() / acc_df[acc_df['model'] == model].total_count.sum()
    print(f'Model: {model} : Acc = {100.0*acc}%')

Model: gpt-3.5-turbo-0125 : Acc = 66.47058823529412%
Model: gpt-3.5-turbo-0125-FC : Acc = 55.76470588235294%


In [10]:
# most of the accuracies are comparable, the biggest difference makers are:
# 1. relevance_score.json: 60% (prompt) vs 0 (FC)
# 2. simple_score.json: 77% (prompt) vs 55% (FC)

In [11]:
error_result_df.head()

,id,model_name,test_category,valid,error,error_type,prompt,model_result_raw,possible_answer,model_result_decoded,filename,model_result,decoded_result
0,7,gpt-3.5-turbo-0125,simple,False,[Invalid syntax. Failed to decode AST. ],ast_decoder:decoder_failed,{'question': 'What are the roots of the quadra...,"[{'solve_quadratic': {'a': 2, 'b': 5, 'c': 3}}]","{'solve_quadratic': {'a': [2], 'b': [5], 'c': ...",NaN,simple_score.json,NaN,NaN
1,14,gpt-3.5-turbo-0125,simple,False,[Nested type checking failed for parameter 'in...,type_error:nested,{'question': 'Calculate the area under the cur...,"[calculate_area_under_curve(function='x**2', i...",{'calculate_area_under_curve': {'function': ['...,[{'calculate_area_under_curve': {'function': '...,simple_score.json,NaN,NaN
2,15,gpt-3.5-turbo-0125,simple,False,[Invalid value for parameter 'function': '3*x*...,value_error:string,{'question': 'Calculate the derivative of the ...,[calculate_derivative(function='3*x**2 + 2*x -...,{'calculate_derivative': {'function': ['3x^2 +...,[{'calculate_derivative': {'function': '3*x**2...,simple_score.json,NaN,NaN
3,16,gpt-3.5-turbo-0125,simple,False,[Invalid syntax. Failed to decode AST. ],ast_decoder:decoder_failed,{'question': 'Calculate the area under the cur...,"[{'name': 'integrate', 'parameters': {'functio...","{'integrate': {'function': ['x^3', 'x**3'], 's...",NaN,simple_score.json,NaN,NaN
4,17,gpt-3.5-turbo-0125,simple,False,[Invalid value for parameter 'function': '2*x*...,value_error:string,{'question': 'Calculate the derivative of the ...,"[calculus.derivative(function='2*x**2', value=...","{'calculus.derivative': {'function': ['2*x^2',...",[{'calculus.derivative': {'function': '2*x**2'...,simple_score.json,NaN,NaN


In [12]:
gpt3_5_fc_relevance_errors = error_result_df[(error_result_df['filename'] == 'relevance_score.json') & (error_result_df['model_name'] == 'gpt-3.5-turbo-0125-FC')]
gpt3_5_fc_relevance_results = full_result_df[(full_result_df['filename'] == 'gorilla_openfunctions_v1_test_relevance_result.json') & (full_result_df['model_name'] == 'gpt-3.5-turbo-0125-FC')]
gpt3_5_prompt_relevance_errors = error_result_df[(error_result_df['filename'] == 'relevance_score.json') & (error_result_df['model_name'] == 'gpt-3.5-turbo-0125')]
gpt3_5_prompt_relevance_results = full_result_df[(full_result_df['filename'] == 'gorilla_openfunctions_v1_test_relevance_result.json') & (full_result_df['model_name'] == 'gpt-3.5-turbo-0125')]

In [13]:
gpt3_5_fc_relevance_errors.shape

(234, 13)

In [14]:
gpt3_5_prompt_relevance_errors.shape

(96, 13)

In [15]:
gpt3_5_fc_relevance_results.shape

(240, 7)

# takeaway - gpt3.5 FC gets relevance results ALMOST ALWAYS WRONG! 234/240 are wrong.

In [16]:
gpt3_5_fc_relevance_errors.head()

,id,model_name,test_category,valid,error,error_type,prompt,model_result_raw,possible_answer,model_result_decoded,filename,model_result,decoded_result
0,1,gpt-3.5-turbo-0125-FC,relevance,False,[Valid syntax. Successfully decode AST when it...,relevance_error:decoder_success,NaN,NaN,NaN,NaN,relevance_score.json,"[{'determine_body_mass_index': '{""weight"": 10,...","[{'determine_body_mass_index': {'weight': 10, ..."
1,2,gpt-3.5-turbo-0125-FC,relevance,False,[Valid syntax. Successfully decode AST when it...,relevance_error:decoder_success,NaN,NaN,NaN,NaN,relevance_score.json,"[{'math_sum': '{""numbers"": [1, 2, 3]}'}, {'mat...","[{'math_sum': {'numbers': [1, 2, 3]}}, {'math_..."
2,3,gpt-3.5-turbo-0125-FC,relevance,False,[Valid syntax. Successfully decode AST when it...,relevance_error:decoder_success,NaN,NaN,NaN,NaN,relevance_score.json,"[{'solve_quadratic_equation': '{""a"": 3, ""b"": -...","[{'solve_quadratic_equation': {'a': 3, 'b': -2..."
3,4,gpt-3.5-turbo-0125-FC,relevance,False,[Valid syntax. Successfully decode AST when it...,relevance_error:decoder_success,NaN,NaN,NaN,NaN,relevance_score.json,"[{'find_critical_points': '{""function"":""3x + 2...",[{'find_critical_points': {'function': '3x + 2...
4,5,gpt-3.5-turbo-0125-FC,relevance,False,[Valid syntax. Successfully decode AST when it...,relevance_error:decoder_success,NaN,NaN,NaN,NaN,relevance_score.json,"[{'find_roots': '{""a"": 0, ""b"": 1, ""c"": 0}'}, {...","[{'find_roots': {'a': 0, 'b': 1, 'c': 0}}, {'f..."


In [17]:
gpt3_5_fc_relevance_results.head()

,idx,result,input_token_count,output_token_count,latency,filename,model_name
0,0,"[{'determine_body_mass_index': '{""weight"": 10,...",101,36,0.799414,gorilla_openfunctions_v1_test_relevance_result...,gpt-3.5-turbo-0125-FC
1,1,"[{'math_sum': '{""numbers"": [1, 2, 3]}'}, {'mat...",112,52,1.101193,gorilla_openfunctions_v1_test_relevance_result...,gpt-3.5-turbo-0125-FC
2,2,"[{'solve_quadratic_equation': '{""a"": 3, ""b"": -...",113,65,1.267069,gorilla_openfunctions_v1_test_relevance_result...,gpt-3.5-turbo-0125-FC
3,3,"[{'find_critical_points': '{""function"":""3x + 2...",133,23,0.650546,gorilla_openfunctions_v1_test_relevance_result...,gpt-3.5-turbo-0125-FC
4,4,"[{'find_roots': '{""a"": 0, ""b"": 1, ""c"": 0}'}, {...",109,61,1.080612,gorilla_openfunctions_v1_test_relevance_result...,gpt-3.5-turbo-0125-FC


In [37]:
# JESUS CHRIST BFCL! There's a bug in your code.
# the ID in the error results is not the same as the idx in the results
# the score.json idx is off by 1 from the idx in the results


# let's see if we can compare FC vs non-FC for these errors
def compare_fc_vs_prompt(fc_errors_df, fc_df, prompt_errors_df, prompt_df, idx=None):
    if idx is None:
        fc_only_errors = set(fc_errors_df.id.values) - set(prompt_errors_df.id.values)
        idx = np.random.choice(list(fc_only_errors))
    print(f'Looking at idx: {idx} (WHICH IS SECRETLY) {idx - 1} in the results.json')
    print(f"Error: {fc_errors_df.model_name.unique()[0]}", fc_errors_df[fc_errors_df['id'] == idx]['error'].item())
    print("FC Model: ", fc_df[fc_df['idx'] == (idx-1)]['result'].item())

    if fc_errors_df[fc_errors_df['id'] == idx].test_category.item() == 'simple':
        # check if fc model has multiple function calls which repeat
        fc_response = fc_df[fc_df['idx'] == (idx-1)]['result'].item()
        print(f"Num FC responses: {len(fc_response)}")
        if len(fc_response) > 1:
            same_bool = [response == fc_response[0] for response in fc_response]
            if sum(same_bool) == len(same_bool):
                print(f"!!! FC model repeated the same function call {len(fc_response)} times. !!!")
            else:
                print(f"FC model had multiple different function calls. Weird.")
        

    print("Prompt Model: ", prompt_df[prompt_df['idx'] == (idx-1)]['result'].item())
    if idx in prompt_errors_df.id.values:
        print("Prompt model also made an error. This is not a clear FC error.")
        print("Prompt Error: ", prompt_errors_df[prompt_errors_df['id'] == idx]['error'].item())
    else:
        print("Prompt model got it right.")

    if fc_errors_df[fc_errors_df['id'] == idx]['test_category'].item() == 'relevance':
        with open("data/gorilla_openfunctions_v1_test_relevance.json", 'r') as f:
            data = [json.loads(line) for line in f.readlines()]
            question_df = pd.DataFrame(data)
    elif fc_errors_df[fc_errors_df['id'] == idx].test_category.item() == 'simple':
        with open("data/gorilla_openfunctions_v1_test_simple.json", 'r') as f:
            data = [json.loads(line) for line in f.readlines()]
            question_df = pd.DataFrame(data)
    else:
        raise ValueError(f"Not implemented for test categories {fc_errors_df[fc_errors_df['id'] == idx].test_category.item()}.")
    
    print("Question: ", question_df.iloc[idx-1]['question'])

In [28]:
compare_fc_vs_prompt(gpt3_5_fc_relevance_errors,
                     gpt3_5_fc_relevance_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_relevance_results,
                     idx=121)

Looking at idx: 121 (WHICH IS SECRETLY) 120 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Valid syntax. Successfully decode AST when it should not.']
FC Model:  [{'caffeine_effect': '{"caffeine_content":95,"drinking_frequency":"daily"}'}]
Prompt Model:  NO tools call.
Prompt model got it right.
Question:  What's the neurological impact of sports on human brain?


In [88]:
# with open("data/gorilla_openfunctions_v1_test_relevance.json", 'r') as f:
#     data = [json.loads(line) for line in f.readlines()]
#     question_df = pd.DataFrame(data)

In [142]:
compare_fc_vs_prompt(gpt3_5_fc_relevance_errors,
                     gpt3_5_fc_relevance_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_relevance_results,
                     idx=113)

Looking at idx: 113 (WHICH IS SECRETLY) 112 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Valid syntax. Successfully decode AST when it should not.']
FC Model:  [{'geocode_address': '{"address":"New York, USA"}'}]
Prompt Model:  [geocode_address(address='New York')]
Prompt model also made an error. This is not a clear FC error.
Prompt Error:  ['Valid syntax. Successfully decode AST when it should not.']
Question:  What's the current traffic condition in New York?


In [150]:
compare_fc_vs_prompt(gpt3_5_fc_relevance_errors,
                     gpt3_5_fc_relevance_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_relevance_results,
                     idx=135
                     )

Looking at idx: 135 (WHICH IS SECRETLY) 134 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Valid syntax. Successfully decode AST when it should not.']
FC Model:  [{'calculate_battle_outcome': '{"battle_name": "World Cup 2022 Final", "strategy_type": "football"}'}, {'calculate_battle_outcome': '{"battle_name": "World Cup 2022 Final", "strategy_type": "football"}'}]
Prompt Model:  [This question does not relate to the available function. No function applies.]
Prompt model got it right.
Question:  Who won the World Cup 2022?


In [66]:
with open("data/gorilla_openfunctions_v1_test_relevance.json", 'r') as f:
    data = [json.loads(line) for line in f.readlines()]
    question_df = pd.DataFrame(data)

question_df.iloc[134].function

{'name': 'calculate_battle_outcome',
 'description': 'Predicts the outcome of a historical battle based on the strategies, army size and other influencing factors.',
 'parameters': {'type': 'dict',
  'properties': {'battle_name': {'type': 'string',
    'description': 'The name of the historical battle.'},
   'strategy_type': {'type': 'string',
    'description': 'The strategy employed in the battle.'},
   'weather_condition': {'type': 'string',
    'description': 'Weather condition during the battle.',
    'default': 'snowing'}},
  'required': ['battle_name', 'strategy_type']}}

In [67]:
# ^^honestly not bad. Pretty innovative way to use an irrelevant function to make it look relevant. Still wrong, but I'm impressed.

# Takeaway: Prompt model understands when the passed functions are irrelevant and says no. But the FC model almost always returns a function call even though it makes no sense and gets Rekt. I suspect this is an issue with the way BFCL is calling the FC model. But will verify after implementing for DBRX

In [19]:
# now compare for simple
gpt3_5_fc_simple_errors = error_result_df[(error_result_df['filename'] == 'simple_score.json') & (error_result_df['model_name'] == 'gpt-3.5-turbo-0125-FC')]
gpt3_5_fc_simple_results = full_result_df[(full_result_df['filename'] == 'gorilla_openfunctions_v1_test_simple_result.json') & (full_result_df['model_name'] == 'gpt-3.5-turbo-0125-FC')]
gpt3_5_prompt_simple_errors = error_result_df[(error_result_df['filename'] == 'simple_score.json') & (error_result_df['model_name'] == 'gpt-3.5-turbo-0125')]
gpt3_5_prompt_simple_results = full_result_df[(full_result_df['filename'] == 'gorilla_openfunctions_v1_test_simple_result.json') & (full_result_df['model_name'] == 'gpt-3.5-turbo-0125')]

In [31]:
compare_fc_vs_prompt(gpt3_5_fc_simple_errors,
                     gpt3_5_fc_simple_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_simple_results,
                     idx=335)

Looking at idx: 335 (WHICH IS SECRETLY) 334 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Wrong number of functions.']
FC Model:  [{'blackjack_check_winner': '{"player_cards": ["A", "10"], "dealer_cards": ["10", "9"], "ace_value": 1}'}, {'blackjack_check_winner': '{"player_cards": ["A", "10"], "dealer_cards": ["10", "9"], "ace_value": 11}'}]
Num FC responses: 2
FC model had multiple different function calls. Weird.
Prompt Model:  [blackjack.check_winner(player_cards=['A', '10'], dealer_cards=['10', '9'], ace_value=1)]
Prompt model got it right.
Question:  Check who is the winner in a game of blackjack given player having A and 10, dealer having 10 and 9. The Ace is considered 1.


In [33]:
compare_fc_vs_prompt(gpt3_5_fc_simple_errors,
                     gpt3_5_fc_simple_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_simple_results,
                     idx=37)

Looking at idx: 37 (WHICH IS SECRETLY) 36 in the results.json
Error: gpt-3.5-turbo-0125-FC ["Invalid value for parameter 'unit': 'mi'. Expected one of ['km', '']. Case insensitive."]
FC Model:  [{'get_shortest_driving_distance': '{"origin":"New York City","destination":"Washington D.C.","unit":"mi"}'}]
Num FC responses: 1
Prompt Model:  [get_shortest_driving_distance(origin='New York City', destination='Washington D.C.')]
Prompt model got it right.
Question:  Find the shortest driving distance between New York City and Washington D.C.


In [34]:
compare_fc_vs_prompt(gpt3_5_fc_simple_errors,
                     gpt3_5_fc_simple_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_simple_results,
                     idx=382)

Looking at idx: 382 (WHICH IS SECRETLY) 381 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Wrong number of functions.']
FC Model:  [{'hilton_hotel_check_availability': '{"location": "Paris", "check_in_date": "2023-04-04", "check_out_date": "2023-04-08", "no_of_adults": 2}'}, {'hilton_hotel_check_availability': '{"location": "Paris", "check_in_date": "2023-04-04", "check_out_date": "2023-04-08", "no_of_adults": 2, "hotel_chain": "Hilton Garden Inn"}'}]
Num FC responses: 2
FC model had multiple different function calls. Weird.
Prompt Model:  [{'name': 'hilton_hotel.check_availability', 'parameters': {'location': 'Paris', 'check_in_date': '2023-04-04', 'check_out_date': '2023-04-08', 'no_of_adults': 2}}]
Prompt model got it right.
Question:  Check if any Hilton Hotel is available for two adults in Paris from April 4th to April 8th?


In [38]:
compare_fc_vs_prompt(gpt3_5_fc_simple_errors,
                     gpt3_5_fc_simple_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_simple_results,
                     idx=381)

Looking at idx: 381 (WHICH IS SECRETLY) 380 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Wrong number of functions.']
FC Model:  [{'hotel_booking': '{"location": "Manhattan, New York", "room_type": "single", "duration": 3, "start_date": "2023-03-10", "preferences": ["pet_friendly"]}'}, {'hotel_booking': '{"location": "Manhattan, New York", "room_type": "single", "duration": 3, "start_date": "2023-03-10", "preferences": ["pet_friendly"]}'}]
Num FC responses: 2
!!! FC model repeated the same function call 2 times. !!!
Prompt Model:  [hotel_booking(location='Manhattan, New York', room_type='single', duration=3, start_date='March 10th, 2023', preferences=['pet_friendly'])]
Prompt model got it right.
Question:  Book a single room at a pet friendly hotel near Manhattan, New York for 3 nights starting from March 10th, 2023.


In [39]:
compare_fc_vs_prompt(gpt3_5_fc_simple_errors,
                     gpt3_5_fc_simple_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_simple_results,
                     idx=383)

Looking at idx: 383 (WHICH IS SECRETLY) 382 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Wrong number of functions.']
FC Model:  [{'book_hotel': '{"hotel_name": "Hilton Hotel", "location": "Chicago", "room_type": "single", "start_date": "10th December 2022", "nights": 2}'}, {'book_hotel': '{"hotel_name": "Hilton Hotel", "location": "Chicago", "room_type": "single", "start_date": "10th December 2022", "nights": 2}'}]
Num FC responses: 2
!!! FC model repeated the same function call 2 times. !!!
Prompt Model:  [book_hotel(hotel_name='Hilton Hotel', location='Chicago', room_type='single', start_date='10th December 2022', nights=2)]
Prompt model got it right.
Question:  Book a single room for two nights at the Hilton Hotel in Chicago, starting from 10th December 2022.


In [2]:
fun_calls = [{'book_hotel': '{"hotel_name": "Hilton Hotel", "location": "Chicago", "room_type": "single", "start_date": "10th December 2022", "nights": 2}'}, {'book_hotel': '{"hotel_name": "Hilton Hotel", "location": "Chicago", "room_type": "single", "start_date": "10th December 2022", "nights": 2}'}]
fun_calls[0] == fun_calls[1]

True

In [42]:
compare_fc_vs_prompt(gpt3_5_fc_simple_errors,
                     gpt3_5_fc_simple_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_simple_results,
                     idx=286)

Looking at idx: 286 (WHICH IS SECRETLY) 285 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Wrong number of functions.']
FC Model:  [{'find_concert': '{"location": "Chicago, IL", "price": 100, "genre": "Rock"}'}, {'find_concert': '{"location": "Chicago, IL", "price": 100, "genre": "Pop"}'}, {'find_concert': '{"location": "Chicago, IL", "price": 100, "genre": "Country"}'}]
Num FC responses: 3
FC model had multiple different function calls. Weird.
Prompt Model:  [find_concert(location='Chicago, IL', price=100, genre='Rock')]
Prompt model got it right.
Question:  Find me a Rock concert in Chicago with ticket availability under $100.


In [43]:
compare_fc_vs_prompt(gpt3_5_fc_simple_errors,
                     gpt3_5_fc_simple_results,
                     gpt3_5_prompt_relevance_errors,
                     gpt3_5_prompt_simple_results,
                     idx=None)

Looking at idx: 128 (WHICH IS SECRETLY) 127 in the results.json
Error: gpt-3.5-turbo-0125-FC ['Wrong number of functions.']
FC Model:  [{'calculate_NPV': '{"cash_flows": [200, 300, 400, 500], "discount_rate": 0.1, "initial_investment": 2000}'}, {'calculate_NPV': '{"cash_flows": [-2000, 200, 300, 400, 500], "discount_rate": 0.1}'}]
Num FC responses: 2
FC model had multiple different function calls. Weird.
Prompt Model:  [calculate_NPV(cash_flows=[200,300,400,500], discount_rate=0.1, initial_investment=2000)]
Prompt model got it right.
Question:  Find the Net Present Value (NPV) of an investment, given cash_flows=[200,300,400,500], a discount rate of 10%, and an initial investment of $2000.


In [45]:
gpt3_5_fc_simple_errors.error.describe()

count                              179
unique                              25
top       [Wrong number of functions.]
freq                               155
Name: error, dtype: object

In [51]:
def check_if_funcall_repeats(fc_errors_df, fc_df, prompt_errors_df, prompt_df, idx=None):
    if idx is None:
        fc_only_errors = set(fc_errors_df.id.values) - set(prompt_errors_df.id.values)
        idx = np.random.choice(list(fc_only_errors))
    # print(f'Looking at idx: {idx} (WHICH IS SECRETLY) {idx - 1} in the results.json')
    # print(f"Error: {fc_errors_df.model_name.unique()[0]}", fc_errors_df[fc_errors_df['id'] == idx]['error'].item())
    # print("FC Model: ", fc_df[fc_df['idx'] == (idx-1)]['result'].item())

    if fc_errors_df[fc_errors_df['id'] == idx].test_category.item() == 'simple':
        # check if fc model has multiple function calls which repeat
        fc_response = fc_df[fc_df['idx'] == (idx-1)]['result'].item()
        # print(f"Num FC responses: {len(fc_response)}")
        if len(fc_response) > 1:
            same_bool = [response == fc_response[0] for response in fc_response]
            if sum(same_bool) == len(same_bool):
                # print(f"!!! FC model repeated the same function call {len(fc_response)} times. !!!")
                return True
            else:
                # print(f"FC model had multiple different function calls. Weird.")
                return False
        
num_fc_repeats = 0
for idx in gpt3_5_fc_simple_errors.id.values:
    if check_if_funcall_repeats(gpt3_5_fc_simple_errors,
                                gpt3_5_fc_simple_results,
                                gpt3_5_prompt_simple_errors,
                                gpt3_5_prompt_simple_results,
                                idx=idx):
        num_fc_repeats += 1
print(f"The FC model repeats function calls {num_fc_repeats}/{gpt3_5_fc_simple_errors.shape[0]} times.")

The FC model repeats function calls 42/179 times.


# so it looks like 155/179 errors are due to the FC model trying multiple function calls when it should have just invoked the one function, once. Sometimes (42 times) it repeats the same function call, but quite often (137 times) it just makes multiple function calls. This is not just a parsing issue, it could be a prompting issue/incorrect way to use the openAI api. Not sure.